## 初期値

In [2]:
import os

os.chdir("/work/app")

# %% 準備
import pandas as pd
from utils.logger import app_logger
from utils.config_loader import get_template_config
from logic.manage.utils.csv_loader import load_all_filtered_dataframes
from logic.controllers.csv_controller import apply_expected_dtypes
from logic.manage.utils.load_template import load_master_and_template
from IPython.display import display
import re
from logic.manage.factory_report import process
from logic.manage.utils.excel_tools import create_label_rows_generic, sort_by_cell_row
from utils.value_setter import set_value_fast, set_value_fast_safe
from logic.manage.utils.summary_tools import summary_apply


# 表示ラベルマップ（処理対象名として使う）
csv_label_map = {"yard": "ヤード一覧", "shipping": "出荷一覧", "receive": "受入一覧"}

debug_receive = "/work/app/data/output/debug_receive.parquet"
debug_shipping = "/work/app/data/output/debug_shipping.parquet"
debug_yard = "/work/app/data/output/debug_yard.parquet"

dfs = {
    "receive": pd.read_parquet(debug_receive),
    "shipping": pd.read_parquet(debug_shipping),
    "yard": pd.read_parquet(debug_yard),
}  # テスト用CSV
# dfs
df_shipping = dfs["shipping"]
df_yard = dfs["yard"]
df_receive = dfs["receive"]

2025-05-08 16:34:25,444 [INFO] (__init__.py:24) [dd3b4088c738/root] ✅ factory_report.py の process 関数を登録しました
2025-05-08 16:34:25,453 [INFO] (__init__.py:24) [dd3b4088c738/root] ✅ balance_sheet.py の process 関数を登録しました
2025-05-08 16:34:25,470 [INFO] (__init__.py:24) [dd3b4088c738/root] ✅ average_sheet.py の process 関数を登録しました
2025-05-08 16:34:25,479 [INFO] (__init__.py:24) [dd3b4088c738/root] ✅ management_sheet.py の process 関数を登録しました
2025-05-08 07:34:25.695 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [3]:
df_yard

,伝票日付,取引先名,品名,正味重量,数量,単位名,単価,金額,営業担当者名,業者CD,業者名,種類CD,種類名,品名CD
0,2025-03-31,ｵﾈｽﾄ･ﾜﾝ,選別,80,80,kg,60.0,"4,800",オネスト・ワン,1308,ｵﾈｽﾄ･ﾜﾝ,1,処分費,30
1,2025-03-31,泉土木,GD,110,110,kg,14.0,"1,540",矢作,3140,ｻﾞｲﾏｯｸｽ,3,有価物,69
2,2025-03-31,泉土木,GC 軽鉄･ｽﾁｰﾙ類,810,810,kg,24.5,"19,845",矢作,9026,ﾚｱﾘﾅ,3,有価物,68
3,2025-03-31,泉宣宏社,GC 軽鉄･ｽﾁｰﾙ類,310,310,kg,0.0,0,渡辺,311,泉宣宏社,3,有価物,68
4,2025-03-31,港区教育委員会事務局学校教育部,混合廃棄物,480,480,kg,45.0,"21,600",小林,923,港区教育委員会事務局学校教育部,1,処分費,14
5,2025-03-31,港区教育委員会事務局学校教育部,混合廃棄物A,130,130,kg,65.0,"8,450",小林,923,港区教育委員会事務局学校教育部,1,処分費,1
6,2025-03-31,五十嵐商会,選別,460,460,kg,65.0,"29,900",渡辺,7996,ｼﾞｪｲｼﾃｨｰ,1,処分費,30
7,2025-03-31,ﾋﾗｶﾜ,GC 軽鉄･ｽﾁｰﾙ類,1470,"1,470",kg,0.0,0,川部,428,ﾋﾗｶﾜ,3,有価物,68
8,2025-03-31,ﾅｲﾃｯｸｽ,選別,1220,"1,220",kg,70.0,"85,400",山本,6509,ﾅｲﾃｯｸｽ,1,処分費,30
9,2025-03-31,ﾃﾙｳｪﾙ東日本,選別(ｷｬﾋﾞﾈｯﾄ),650,650,kg,65.0,"42,250",渡辺,4418,ﾃﾙｳｪﾙ東日本,1,処分費,171


In [4]:
df_shipping

,伝票日付,出荷番号,取引先名,業者CD,業者名,現場CD,現場名,品名,正味重量,数量,単位名,単価,金額,運搬業者名,伝票区分名
0,2025-03-31,10499,■I ･ S ｱｲ･ｴｽ,8327,丸源起業,0,丸源起業ひかりｴｺｽﾃｰｼｮﾝ,混合廃棄物(A),13870,"13,870",kg,22,"305,140",ｵﾈｽﾄ,支払
1,2025-03-31,10542,東京ﾎﾞｰﾄﾞ工業,5238,東京ﾎﾞｰﾄﾞ工業,0,東京ﾎﾞｰﾄﾞ工場新木場ﾘｻｲｸﾘﾝｸﾞ工場,木くず,2620,"2,620",kg,7,"18,340",東亜物流,支払
2,2025-03-31,10543,ｵﾈｽﾄ,1,ｵﾈｽﾄ,2,ｵﾈｽﾄ新木場ﾔｰﾄﾞ,金属くず ｼｭﾚｯﾀ,730,730,kg,0,0,■ｴｺﾗｲﾝ,売上
3,2025-03-31,10545,ｵﾈｽﾄ,1,ｵﾈｽﾄ,2,ｵﾈｽﾄ新木場ﾔｰﾄﾞ,鉄千地,1460,"1,460",kg,0,0,東亜物流,支払
4,2025-03-31,10546,宮崎,747,宮崎,1,宮崎新木場リサイクルセンター,ﾀﾞﾝﾎﾞｰﾙ,250,250,kg,14,"3,500",■ｴｺﾗｲﾝ,売上
5,2025-03-31,10547,東京ﾎﾞｰﾄﾞ工業,5238,東京ﾎﾞｰﾄﾞ工業,0,東京ﾎﾞｰﾄﾞ工場新木場ﾘｻｲｸﾘﾝｸﾞ工場,木くず,2050,"2,050",kg,7,"14,350",■ｴｺﾗｲﾝ,支払
6,2025-03-31,10548,ADVANCE,8327,丸源起業,0,丸源起業ひかりｴｺｽﾃｰｼｮﾝ,混合廃棄物(A),0,"13,850",kg,1,"13,850",ｵﾈｽﾄ,支払
7,2025-03-31,10541,東京産資源,8425,東京産資源,0,東京産資源,GB(ｷﾞﾛA),500,500,kg,33.5,"16,750",ｵﾈｽﾄ,売上
8,2025-03-31,10549,ADVANCE,8327,丸源起業,0,丸源起業ひかりｴｺｽﾃｰｼｮﾝ,混合廃棄物(B),0,"10,590",kg,1,"10,590",ｵﾈｽﾄ,支払
9,2025-03-31,10551,東京産資源,8425,東京産資源,1,【ﾔｰﾄﾞ】,GB(ｷﾞﾛA),3810,"3,810",kg,31.5,"120,015",東京産資源,売上


In [5]:
df_receive

,伝票日付,売上日付,支払日付,業者CD,業者名,伝票区分CD,伝票区分名,品名CD,品名,正味重量,数量,単位CD,単位名,単価,金額,受入番号,集計項目CD,集計項目
0,2025-03-31,2025/03/31(月),2025/03/31(月),10,ｸﾛｻﾜ工業,1,売上,2,混合廃棄物B,640,640,3,kg,70,44800.0,46663,3,C（解体）
1,2025-03-31,2025/03/31(月),2025/03/31(月),5515,ｷｬﾌﾟﾃｨｿﾘｭｰｼｮﾝｽﾞ,1,売上,350,運搬費(4ｔﾕﾆｯｸ車),0,1,7,台,"30,000",30000.0,46660,4,D（建設）
2,2025-03-31,2025/03/31(月),2025/03/31(月),5515,ｷｬﾌﾟﾃｨｿﾘｭｰｼｮﾝｽﾞ,2,支払,68,GC 軽鉄･ｽﾁｰﾙ類,480,480,3,kg,15.5,7440.0,46660,4,D（建設）
3,2025-03-31,2025/03/31(月),2025/03/31(月),5676,千代田電興,1,売上,1,混合廃棄物A,300,300,3,kg,65,19500.0,46674,4,D（建設）
4,2025-03-31,2025/03/31(月),2025/03/31(月),5854,沖原工務所,1,売上,2,混合廃棄物B,70,70,3,kg,70,4900.0,46743,3,C（解体）
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,2025-03-31,2025/03/31(月),2025/03/31(月),1656,真和商事,1,売上,93,混合廃棄物(ﾋﾞﾝ･缶･ﾍﾟｯﾄﾎﾞﾄﾙ),120,120,3,kg,45,5400.0,46625,2,B（残置）
191,2025-03-31,2025/03/31(月),2025/03/31(月),1656,真和商事,1,売上,93,混合廃棄物(ﾋﾞﾝ･缶･ﾍﾟｯﾄﾎﾞﾄﾙ),140,140,3,kg,45,6300.0,46634,2,B（残置）
192,2025-03-31,2025/03/31(月),2025/03/31(月),2246,東ｿｰ物流,1,売上,1,混合廃棄物A,730,730,3,kg,60,43800.0,46635,2,B（残置）
193,2025-03-31,2025/03/31(月),2025/03/31(月),955,CRASHIP,1,売上,2,混合廃棄物B,380,380,3,kg,70,26600.0,46710,3,C（解体）


### Process

In [6]:
# 処理の統合
def process1(dfs: dict) -> pd.DataFrame:
    logger = app_logger()
    """
    Streamlitの選択に基づき、工場日報（処分パート）を処理するエントリーポイント関数。
    """

    logger = app_logger()

    # --- テンプレート設定の取得 ---
    template_key = "balance_sheet"
    template_config = get_template_config()[template_key]
    template_name = template_config["key"]
    csv_keys = template_config["required_files"]
    logger.info(f"[テンプレート設定読込] key={template_key}, files={csv_keys}")

    # --- CSVの調整・読み込み ---
    dfs = apply_expected_dtypes(dfs, template_key)
    df_dict = load_all_filtered_dataframes(dfs, csv_keys, template_name)
    df_receive = df_dict.get("receive")
    df_shipping = df_dict.get("shipping")
    df_yard = df_dict.get("yard")

    return df_receive, df_yard, df_shipping


df_receive, df_yard, df_shipping = process1(dfs)

2025-05-08 16:34:25,907 [INFO] (2821753602.py:15) [dd3b4088c738/root] [テンプレート設定読込] key=balance_sheet, files=['receive', 'yard', 'shipping']
2025-05-08 16:34:25,964 [INFO] (csv_controller.py:42) [dd3b4088c738/root] ✅ 型を適用しました: receive
2025-05-08 16:34:25,990 [INFO] (csv_controller.py:42) [dd3b4088c738/root] ✅ 型を適用しました: shipping


2025-05-08 16:34:26,013 [INFO] (csv_controller.py:42) [dd3b4088c738/root] ✅ 型を適用しました: yard


In [7]:
display(df_receive.head())
display(df_yard.head())
display(df_shipping.head())

,金額,受入番号,数量,正味重量,単位名,単価,伝票区分名,品名,伝票日付
0,44800,46663,640,640,kg,70,売上,混合廃棄物B,2025-03-31
1,30000,46660,1,0,台,30000,売上,運搬費(4ｔﾕﾆｯｸ車),2025-03-31
2,7440,46660,480,480,kg,15,支払,GC 軽鉄･ｽﾁｰﾙ類,2025-03-31
3,19500,46674,300,300,kg,65,売上,混合廃棄物A,2025-03-31
4,4900,46743,70,70,kg,70,売上,混合廃棄物B,2025-03-31


,種類名,数量,正味重量,品名
0,処分費,80,80,選別
1,有価物,110,110,GD
2,有価物,810,810,GC 軽鉄･ｽﾁｰﾙ類
3,有価物,310,310,GC 軽鉄･ｽﾁｰﾙ類
4,処分費,480,480,混合廃棄物


,業者CD,業者名,金額,正味重量,品名
0,8327,丸源起業,305140,13870,混合廃棄物(A)
1,5238,東京ﾎﾞｰﾄﾞ工業,18340,2620,木くず
2,1,ｵﾈｽﾄ,0,730,金属くず ｼｭﾚｯﾀ
3,1,ｵﾈｽﾄ,0,1460,鉄千地
4,747,宮崎,3500,250,ﾀﾞﾝﾎﾞｰﾙ


# 工場搬出入

## プロセス

In [44]:
from logic.manage.processors.balance_sheet.balance_sheet_fact import (
    process_factory_report,
)
from logic.manage.processors.balance_sheet.balance_sheet_syobun import (
    calculate_total_disposal_cost,
)
from logic.manage.processors.balance_sheet.balance_sheet_yuukabutu import calculate_total_valuable_material_cost
from logic.manage.processors.balance_sheet.balance_sheet_inbound_truck_count import inbound_truck_count
from logic.manage.processors.balance_sheet.balacne_sheet_inbound_weight import inbound_weight
from logic.manage.processors.balance_sheet.balance_sheet_honest import calculate_honest_sales_by_unit
from logic.manage.processors.balance_sheet.balance_sheet_yuka_kaitori import calculate_purchase_value_of_valuable_items
from logic.manage.processors.balance_sheet.balance_sheet_etc import calculate_misc_summary_rows

# 処理の統合
def process(dfs: dict) -> pd.DataFrame:
    logger = app_logger()

    # --- ① マスターCSVの読み込み ---
    config = get_template_config()["balance_sheet"]
    master_path = config["master_csv_path"]["factory"]
    master_csv = load_master_and_template(master_path)

    # --- テンプレート設定の取得 ---
    template_key = "balance_sheet"
    template_config = get_template_config()[template_key]
    template_name = template_config["key"]
    csv_keys = template_config["required_files"]
    logger.info(f"[テンプレート設定読込] key={template_key}, files={csv_keys}")

    # --- CSVの読み込み ---
    df_dict = load_all_filtered_dataframes(dfs, csv_keys, template_name)
    df_receive = df_dict.get("receive")
    df_shipping = df_dict.get("shipping")
    df_yard = df_dict.get("yard")

    # --- 個別処理 ---
    logger.info("▶️ 搬出量データ処理開始")
    master_csv = process_factory_report(dfs, master_csv)

    logger.info("▶️ 処分費データ処理開始")
    master_csv.loc[master_csv["大項目"] == "処分費", "値"] = (
        calculate_total_disposal_cost(df_yard, df_shipping)
    )

    logger.info("▶️ 有価物データ処理開始")
    master_csv.loc[master_csv["大項目"] == "有価物", "値"] = (
        calculate_total_valuable_material_cost(df_yard, df_shipping)
    )

    logger.info("▶️ 搬入台数データ処理開始")
    master_csv.loc[master_csv["大項目"] == "搬入台数", "値"] = (
        inbound_truck_count(df_receive)
    )

    logger.info("▶️ 搬入量データ処理開始")
    master_csv.loc[master_csv["大項目"] == "搬入量", "値"] = (
        inbound_weight(df_receive)
    )

    logger.info("▶️ オネストkg_m3データ処理開始")
    master_csv.loc[master_csv["大項目"] == "オネストkg", "値"],master_csv.loc[master_csv["大項目"] == "オネストm3", "値"] = (
        calculate_honest_sales_by_unit(df_receive)
    )

    
    logger.info("▶️ 有価買取データ処理開始")
    master_csv.loc[master_csv["大項目"] == "有価買取", "値"] = (
        calculate_purchase_value_of_valuable_items(df_receive)
    )

    # 合計などのその他処理
    master_csv= calculate_misc_summary_rows(master_csv, df_receive)

    return master_csv


master_csv = process(dfs)
master_csv

2025-05-08 17:01:25,082 [INFO] (1393764774.py:28) [dd3b4088c738/root] [テンプレート設定読込] key=balance_sheet, files=['receive', 'yard', 'shipping']
2025-05-08 17:01:25,116 [INFO] (1393764774.py:37) [dd3b4088c738/root] ▶️ 搬出量データ処理開始
2025-05-08 17:01:25,144 [INFO] (factory_report.py:27) [dd3b4088c738/root] [テンプレート設定読込] key=factory_report, files=['yard', 'shipping']
2025-05-08 17:01:25,166 [INFO] (factory_report.py:35) [dd3b4088c738/root] ▶️ 出荷処分データ処理開始
/work/app/utils/data_cleaning.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['8339' '5238' '8340' '8085' '6421' '8337' '8215' '8336' '8327' '8327'
 '8327' '8327' '705' '1185' '8076' '8077' '5199' '8334' '6797' '8338'
 '8341' '8080' '349' '6786' '5227' '8329' '8360' '8363' '8398' '8679'
 '339' '8082']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[valid, col] = df.loc[valid, col].apply(lambda x: str(int(float(x)

,大項目,セル,値
0,搬出量,E6,101130.0
1,処分費,E7,1689200
2,有価物,E8,7535
3,搬入台数,B5,119
4,搬入量,B6,84180
5,オネストkg,B7,5029470.0
6,オネストm3,B8,294000.0
7,有価買取,B9,98115
8,月日,D2,R7年3月31日
9,売上計,B12,5225355


## 排出量（工場日報から）

In [9]:
from utils.value_setter import set_value_fast_safe


def reflect_total_from_factory(master_csv, df_factory):

    total_sum = df_factory.loc[df_factory["大項目"] == "総合計", "値"].squeeze()

    match_columns = ["大項目"]
    match_value = ["搬出量"]
    master_csv = set_value_fast_safe(master_csv, match_columns, match_value, total_sum)

    return master_csv

In [10]:
def process_factory_report(dfs, master_csv):
    logger = app_logger()
    from logic.manage.factory_report import process

    # 工場日報からdfを読込
    df_factory = process(dfs)

    # 搬出量を抜出
    after_master_csv = reflect_total_from_factory(master_csv, df_factory)

    return after_master_csv

## 仕入処分費

### 処分費

In [11]:
# 業者別の処分費
def sum_syobun_cost():
    syobun_cost = shobun_cost(df_shipping)
    sum_gyosya = int(syobun_cost["値"].sum())

    # df_shippingの金庫費
    master_kinko = shobun_cost_kinko(df_shipping)
    sum_kinko = int(master_kinko["値"].sum())
    sum_kinko

    # df_yardの金庫費
    master_kinko_yard = shobun_cost_yard(df_yard)
    sum_kinko_yard = int(master_kinko_yard["値"].sum())
    sum_kinko_yard

    # それぞれのmaster_csvを合計し、処分費とする
    sum_syobun_cost = sum_gyosya + sum_kinko + sum_kinko_yard
    sum_syobun_cost

    return sum_syobun_cost

In [12]:
def shobun_cost(df_shipping):

    # --- ① マスターCSVの読み込み ---
    config = get_template_config()["balance_sheet"]
    master_path = config["master_csv_path"]["shobun_cost"]
    master_csv = load_master_and_template(master_path)

    # # 👇 業者CDを文字列に揃える
    df_shipping["業者CD"] = df_shipping["業者CD"].astype(str)
    master_csv["業者CD"] = master_csv["業者CD"].astype(str)

    # 処分費の総計
    key_cols = ["業者CD"]
    source_col = "金額"
    master_csv = summary_apply(
        master_csv,
        df_shipping,
        key_cols,
        source_col,
    )
    return master_csv

### 処分費_金庫

In [13]:
from utils.config_loader import get_unit_price_table_csv
from logic.manage.utils.multiply_tools import multiply_columns


def shobun_cost_kinko(df_shipping: pd.DataFrame) -> pd.DataFrame:
    """
    出荷データから処分コストを計算する関数

    Parameters
    ----------
    df_shipping : pd.DataFrame
        出荷データのDataFrame

    Returns
    -------
    pd.DataFrame
        単価×正味重量の「値」を含むマスターDataFrame
    """

    # --- ① マスターCSVの読み込み ---
    config = get_template_config()["balance_sheet"]
    master_path = config["master_csv_path"]["syobun_cost_kinko"]
    master_csv = load_master_and_template(master_path)

    # --- ② 正味重量の更新 ---
    key_cols = ["業者名", "品名"]
    source_col = "正味重量"
    master_with_weight = summary_apply(
        master_csv,
        df_shipping,
        key_cols,
        source_col,
        source_col,  # target_colとsource_colが同じ
    )

    # --- ③ 単価の設定 ---
    unit_price_csv = get_unit_price_table_csv()
    master_with_price = summary_apply(
        master_with_weight,
        unit_price_csv,
        key_cols,
        source_col="設定単価",
        target_col="設定単価",
    )

    # --- ④ 単価×正味重量の計算 ---
    result_df = multiply_columns(
        master_with_price, col1="設定単価", col2="正味重量", result_col="値"
    )

    return result_df

### 処分費_金庫_ヤード

In [14]:
from utils.config_loader import get_unit_price_table_csv
from logic.manage.utils.multiply_tools import multiply_columns


def shobun_cost_yard(df_yard: pd.DataFrame) -> pd.DataFrame:
    """
    Yardデータ（ヤード集荷）から処分コストを計算する関数。

    マスターCSVとYard出荷データをマージし、
    正味重量および単価を適用した上で、
    最終的な「単価 × 正味重量」の金額列（値）を計算する。

    Parameters
    ----------
    df_yard : pd.DataFrame
        Yard出荷データのDataFrame。

    Returns
    -------
    pd.DataFrame
        単価・正味重量・および「単価 × 正味重量（値）」を含む更新後のマスターDataFrame。
    """

    # --- ① マスターCSVの読み込み ---
    config = get_template_config()["balance_sheet"]
    master_path = config["master_csv_path"]["syobun_cost_kinko_yard"]
    master_csv = load_master_and_template(master_path)

    # --- ② 正味重量の更新 ---
    key_cols = ["種類名", "品名"]
    source_col = "正味重量"
    master_with_weight = summary_apply(
        master_csv,
        df_yard,
        key_cols,
        source_col,
        source_col,  # target_colとsource_colが同じ
    )

    # --- ③ 単価の設定 ---
    unit_price_csv = get_unit_price_table_csv()
    master_with_price = summary_apply(
        master_with_weight,
        unit_price_csv,
        key_cols,
        source_col="設定単価",
        target_col="設定単価",
    )

    # --- ④ 単価×正味重量の計算 ---
    result_df = multiply_columns(
        master_with_price, col1="設定単価", col2="正味重量", result_col="値"
    )

    return result_df

### 処分費合計

In [15]:
master_csv1

NameError: name 'master_csv1' is not defined

In [ ]:
def sum_shobun(df_shobun, df_kinko, df_kinko_yard):

    # 列名を変更
    df_shobun = df_shobun.rename(columns={"業者CD": "大項目"})
    sum_shobun_total = df_shobun["値"].sum()

    # 合計
    match_columns = ["大項目"]
    match_value = ["処分費"]
    master_csv2 = set_value_fast_safe(
        master_csv1, match_columns, match_value, sum_shobun_total
    )

    return master_csv2


df_shobun = shobun_cost(df_shipping)
df_kinko = shobun_cost_kinko(df_shipping)
df_kinko_yard = shobun_cost_yard(df_yard)

master_csv = sum_shobun(df_shobun, df_kinko, df_kinko_yard)
master_csv

/tmp/ipykernel_273019/3117396381.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_shipping["業者CD"] = df_shipping["業者CD"].astype(str)
2025-05-02 15:42:31,661 [INFO] (summary_tools.py:78) [dd3b4088c738/root] ▶️ マスター更新処理: キー=['業者CD'], 集計列=金額 ➡ 書き込み列=値
2025-05-02 15:42:31,730 [INFO] (summary_tools.py:78) [dd3b4088c738/root] ▶️ マスター更新処理: キー=['業者名', '品名'], 集計列=正味重量 ➡ 書き込み列=正味重量
2025-05-02 15:42:31,754 [INFO] (summary_tools.py:78) [dd3b4088c738/root] ▶️ マスター更新処理: キー=['業者名', '品名'], 集計列=設定単価 ➡ 書き込み列=設定単価
2025-05-02 15:42:31,785 [INFO] (summary_tools.py:78) [dd3b4088c738/root] ▶️ マスター更新処理: キー=['種類名', '品名'], 集計列=正味重量 ➡ 書き込み列=正味重量
2025-05-02 15:42:31,812 [INFO] (summary_tools.py:78) [dd3b4088c738/root] ▶️ マスター更新処理: キー=['種類名', '品名'], 集計列=設定単価 ➡ 書き込み列=設

,大項目,セル,値
0,搬出量,E6,101130.0
1,処分費,E7,1673600.0


## 有価物

In [ ]:
def calculate_total_valuable_material_cost(
    df_shipping: pd.DataFrame, df_yard: pd.DataFrame
) -> int:
    """
    出荷データとヤードデータをもとに、有価物の合計金額を算出する。

    処理内容：
    - 出荷データから業者別に有価金額を集計
    - ヤードデータから品目別に有価金額を集計
    - それぞれの「値」を合算し、総額を返す

    Parameters
    ----------
    df_shipping : pd.DataFrame
        出荷一覧（業者名、金額を含む）

    df_yard : pd.DataFrame
        ヤード出荷データ（品名、数量を含む）

    Returns
    -------
    int
        出荷分 + ヤード分の有価物合計金額
    """

    # --- 有価_出荷 ---
    shipping_summary_df = aggregate_valuable_material_by_vendor(df_shipping)
    sum_shipping = shipping_summary_df["値"].sum()

    # --- 有価_ヤード ---
    yard_summary_df = calculate_valuable_material_cost_by_item(df_yard)
    sum_yard = yard_summary_df["値"].sum()

    # --- 合計 ---
    total_value = int(sum_shipping + sum_yard)
    return total_value


sum(df_yard, df_shipping)

2025-05-08 15:05:56,736 [INFO] (summary_tools.py:80) [dd3b4088c738/root] ▶️ マスター更新処理: キー=['業者名'], 集計列=金額 ➡ 書き込み列=値
2025-05-08 15:05:56,773 [INFO] (summary_tools.py:80) [dd3b4088c738/root] ▶️ マスター更新処理: キー=['品名'], 集計列=数量 ➡ 書き込み列=数量
2025-05-08 15:05:56,794 [INFO] (summary_tools.py:80) [dd3b4088c738/root] ▶️ マスター更新処理: キー=['品名'], 集計列=設定単価 ➡ 書き込み列=設定単価


4070.0
3465.0


7535

### 有価物_出荷

In [ ]:
def aggregate_valuable_material_by_vendor(shipping_df: pd.DataFrame) -> pd.DataFrame:
    """
    出荷データをもとに、有価物（業者別）の金額を集計してマスターに反映する。

    Parameters
    ----------
    shipping_df : pd.DataFrame
        出荷データ（業者名、金額を含む）

    Returns
    -------
    pd.DataFrame
        「業者名」ごとの金額を「値」列に反映したマスターDataFrame
    """

    # --- ① マスターCSVの読み込み ---
    config = get_template_config()["balance_sheet"]
    master_path = config["master_csv_path"]["yuka_shipping"]
    master_df = load_master_and_template(master_path)

    # --- ② 業者別に金額を集計し、「値」列に反映 ---
    key_cols = ["業者名"]
    aggregated_df = summary_apply(
        master_df,
        shipping_df,
        key_cols=key_cols,
        source_col="金額",
        target_col="値",
    )

    return aggregated_df

### 有価ヤード

In [ ]:
from utils.config_loader import get_unit_price_table_csv
from logic.manage.utils.multiply_tools import multiply_columns


def calculate_valuable_material_cost_by_item(df_yard: pd.DataFrame) -> pd.DataFrame:
    """
    出荷データと単価表から、有価物（品名別）の金額を算出する。

    処理ステップ:
    1. 出荷データから品名ごとの数量をマスターに集計適用
    2. 単価テーブルから品名ごとの設定単価を適用
    3. 数量 × 単価で金額（値）を計算し、大項目列に品名を転記

    Parameters
    ----------
    df_yard : pd.DataFrame
        ヤードデータ（品名、数量を含む）

    Returns
    -------
    pd.DataFrame
        「品名（→大項目）」「数量」「設定単価」「値（金額）」を含む集計済みデータ
    """

    # --- ① マスターCSVの読み込み ---
    config = get_template_config()["balance_sheet"]
    master_path = config["master_csv_path"]["yuka_yard"]
    master_df = load_master_and_template(master_path)

    # --- ② 品名ごとの数量を集計して反映 ---
    master_with_quantity = summary_apply(
        master_df,
        df_yard,
        key_cols=["品名"],
        source_col="数量",
        target_col="数量",
    )

    # --- ③ 単価テーブルから設定単価を反映（有価物のみ） ---
    unit_price_df = get_unit_price_table_csv()
    unit_price_df = unit_price_df[unit_price_df["必要項目"] == "有価物"]

    master_with_unit_price = summary_apply(
        master_with_quantity,
        unit_price_df,
        key_cols=["品名"],
        source_col="設定単価",
        target_col="設定単価",
    )

    # --- ④ 数量 × 単価 → 値 を計算 ---
    result_df = multiply_columns(
        master_with_unit_price, col1="設定単価", col2="数量", result_col="値"
    )

    # --- ⑤ 品名列 → 大項目列へコピー（用途別での出力整形） ---
    result_df = result_df.rename(columns={"品名": "大項目"})

    return result_df

## 売上

### 搬入台数

In [ ]:
def inbound_truck_count(df_receive):
    total = df_receive["受入番号"].nunique()
    return total

119

### 搬入量

In [ ]:
def inbound_weight(df_receive):
    total = int(df_receive["正味重量"].sum())
    return total

84180

### オネスト/kg　and オネストm3

,項目,伝票区分名,単位名,値
0,オネストkg,売上,NaN,NaN
1,オネストm3,売上,m3,NaN


### 有価買取

In [ ]:
def calculate_purchase_value_of_valuable_items(receive_df: pd.DataFrame) -> int:
    """
    有価物（買取）の買取金額合計を算出する。

    処理ステップ：
    1. 「品名 × 単価」ごとの数量を受入データから集計
    2. 単価マスタ（有価買取）を反映
    3. 単価 × 数量 → 値（金額）を算出
    4. 「支払」の伝票区分行に、受入データからの実金額を反映
    5. 全体の合計金額を返す

    Parameters
    ----------
    receive_df : pd.DataFrame
        受入データ（品名、単価、数量、金額、伝票区分名などを含む）

    Returns
    -------
    int
        有価物買取の総額（金額の合計）
    """

    # --- ① マスターCSVの読み込み ---
    config = get_template_config()["balance_sheet"]
    master_path = config["master_csv_path"]["uriage_yuka_kaitori"]
    master_df = load_master_and_template(master_path)

    # --- ② 「品名 × 単価」で数量を集計して反映 ---
    master_with_quantity = summary_apply(
        master_df,
        receive_df,
        key_cols=["品名", "単価"],
        source_col="数量",
        target_col="数量"
    )

    # --- ③ 単価マスタ（有価買取）の適用 ---
    unit_price_df = get_unit_price_table_csv()
    unit_price_df = unit_price_df[unit_price_df["必要項目"] == "有価買取"]

    master_with_prices = summary_apply(
        master_with_quantity,
        unit_price_df,
        key_cols=["品名", "単価"],
        source_col="設定単価",
        target_col="設定単価"
    )

    # --- ④ 単価 × 数量 を計算して金額（値）を算出 ---
    result_df = multiply_columns(
        master_with_prices,
        col1="設定単価",
        col2="数量",
        result_col="値"
    )

    # --- ⑤ 「支払」行に実際の金額を反映 ---
    payment_summary = summary_apply(
        master_with_prices,
        receive_df,
        key_cols=["伝票区分名"],
        source_col="金額",
        target_col="値"
    )

    if not payment_summary[payment_summary["伝票区分名"] == "支払"].empty:
        payment_value = payment_summary.loc[payment_summary["伝票区分名"] == "支払", "値"].values[0]
        result_df.loc[result_df["伝票区分名"] == "支払", "値"] = payment_value

    # --- ⑥ 合計金額を返す ---
    total = int(result_df["値"].sum())
    return total

,伝票区分名,単位名,品名,単価,値
0,支払,kg,NaN,NaN,NaN
1,NaN,NaN,ﾗｼﾞｴﾀｰ,0.0,NaN
2,NaN,NaN,ｷﾞﾛ材,0.0,NaN
3,NaN,NaN,銅,0.0,NaN
4,NaN,NaN,ﾄﾗﾝｽ,0.0,NaN
5,NaN,NaN,給湯器,0.0,NaN
6,NaN,NaN,鉄千地,0.0,NaN
7,NaN,NaN,配線,0.0,NaN
8,NaN,NaN,ｱﾙﾐ類,0.0,NaN
9,NaN,NaN,ｽﾃﾝﾚｽ,0.0,NaN


### その他処理

In [ ]:
from utils.date_tools import to_reiwa_format
from utils.config_loader import get_template_config
from logic.manage.utils.template_loader import load_master_and_template
from logic.manage.utils.summary_tools import summary_apply, set_value_fast_safe
import pandas as pd


def calculate_misc_summary_rows(master_csv: pd.DataFrame, df_receive: pd.DataFrame) -> pd.DataFrame:
    """
    売上・仕入・損益の補足行（「月日」「売上計」「仕入計」「損益」）を計算し、マスターに追加する。

    Parameters
    ----------
    summary_df : pd.DataFrame
        処分費・有価物・オネスト売上などの金額が格納されたマスターDataFrame

    df_receive : pd.DataFrame
        受入データ（伝票日付を含む）

    Returns
    -------
    pd.DataFrame
        summary_df に対して「etc」マスタ内容（補足行）を結合したDataFrame
    """

    # --- ① etcマスターCSVの読み込み ---
    config = get_template_config()["balance_sheet"]
    master_path = config["master_csv_path"]["etc"]
    etc_df = load_master_and_template(master_path)

    # --- ② 日付を和暦で記載（最初の受入日） ---
    first_invoice_date = pd.to_datetime(df_receive["伝票日付"].values[0]).date()
    reiwa_date = to_reiwa_format(first_invoice_date)
    etc_df = set_value_fast_safe(
        df=etc_df,
        match_columns=["大項目"],
        match_values=["月日"],
        value=reiwa_date,
        value_col="値"
    )

    # --- ③ 売上合計（オネストkg + オネストm3 - 有価買取） ---
    honest_kg = int(master_csv.loc[master_csv["大項目"] == "オネストkg", "値"].values[0])
    honest_m3 = int(master_csv.loc[master_csv["大項目"] == "オネストm3", "値"].values[0])
    yuka_kaitori = int(master_csv.loc[master_csv["大項目"] == "有価買取", "値"].values[0])
    sales_total = honest_kg + honest_m3 - yuka_kaitori
    etc_df = set_value_fast_safe(etc_df, ["大項目"], ["売上計"], sales_total, "値")

    # --- ④ 仕入合計（処分費 - 有価物） ---
    shobun_cost = int(master_csv.loc[master_csv["大項目"] == "処分費", "値"].values[0])
    yuka_cost = int(master_csv.loc[master_csv["大項目"] == "有価物", "値"].values[0])
    cost_total = shobun_cost - yuka_cost
    etc_df = set_value_fast_safe(etc_df, ["大項目"], ["仕入計"], cost_total, "値")

    # --- ⑤ 損益（売上計 - 仕入計） ---
    profit_total = sales_total - cost_total
    etc_df = set_value_fast_safe(etc_df, ["大項目"], ["損益"], profit_total, "値")

    # --- ⑥ summary_df に etc_df を結合して返す ---
    result_df = pd.concat([master_csv, etc_df], axis=0, ignore_index=True)

    return result_df


,大項目,セル,値
0,月日,D2,R7年3月31日
1,売上計,B12,NaN
2,仕入計,E12,NaN
3,損益,G12,NaN
